In [1]:
from os import sys
sys.path.append("../")

In [22]:
from utils import get_dataloader
from embedding import setup_vae, get_dataloader
import torch
import torch.nn as nn
from torchvision import models

In [7]:
vae, _ = setup_vae("utkface")
vae.load_state_dict(torch.load("../saved_models/utkface_beta1_vae.pt"))

utkface


<All keys matched successfully>

In [17]:
vae = vae.cuda()

In [9]:
train_loader, test_loader = get_dataloader("utkface")

In [12]:
model = nn.Sequential(nn.Linear(32, 10), nn.ReLU(), nn.Linear(10, 5)).cuda()

In [13]:
optim = torch.optim.Adam(model.parameters(), lr=3e-4)

In [21]:
for e in range(20):
    for idx, (data, labels, _) in enumerate(train_loader):
        data, labels = data.cuda(), labels.cuda()
        with torch.no_grad():
            z, _ = vae.encode(data)
        preds = model(z)
        loss = nn.CrossEntropyLoss()(preds, labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if idx % 20 == 0:
            print("{:.4f}".format(loss.item()))
    correct, total = 0, 0
    for idx, (data, labels, _) in enumerate(test_loader):
        data, labels = data.cuda(), labels.cuda()
        with torch.no_grad():
            z, _ = vae.encode(data)
        preds = model(z)
        correct += (preds.argmax(dim=1) == labels).sum()
        total += data.shape[0]
    print(correct / total)

1.5293
1.4980
1.5101
1.4891
1.4848
corrupt label
tensor(0.4597, device='cuda:0')
1.4776
corrupt label
1.4677
1.4581
1.4287
1.4720
tensor(0.4696, device='cuda:0')
1.4598
1.4026
1.4045
1.3592


KeyboardInterrupt: 

In [29]:
full_model = models.resnet18()
full_model.fc = nn.Linear(512, 5)

In [30]:
full_model = full_model.cuda()
optim = torch.optim.Adam(full_model.parameters(), lr=3e-4)

In [31]:
for e in range(20):
    for idx, (data, labels, _) in enumerate(train_loader):
        data, labels = data.cuda(), labels.cuda()
        with torch.no_grad():
            x, _, _, _ = vae(data)
        preds = full_model(data)
        loss = nn.CrossEntropyLoss()(preds, labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if idx % 20 == 0:
            print("{:.4f}".format(loss.item()))
    correct, total = 0, 0
    for idx, (data, labels, _) in enumerate(test_loader):
        data, labels = data.cuda(), labels.cuda()
        with torch.no_grad():
            x, _, _, _ = vae(data)
        preds = full_model(data)
        correct += (preds.argmax(dim=1) == labels).sum()
        total += data.shape[0]
    print("acc ", correct / total)

1.5687
corrupt label
0.9121
0.7499
0.8639
0.7068
acc  tensor(0.7379, device='cuda:0')
0.7495
0.5467
0.5960
0.6124
0.6049
corrupt label
acc  tensor(0.7704, device='cuda:0')
0.3897
0.4502
0.4503
corrupt label
0.5756
0.5327
acc  tensor(0.7805, device='cuda:0')
0.2889
0.2435
0.3699
0.4549
corrupt label
0.3705
acc  tensor(0.7693, device='cuda:0')
0.2317
0.1636
0.2723
0.2922
corrupt label
0.3062
acc  tensor(0.7505, device='cuda:0')
0.2278
0.2295
corrupt label
0.1122
0.1757
0.3009
acc  tensor(0.7615, device='cuda:0')
0.1764
0.1352
0.1476
0.1177
corrupt label
0.2180
acc  tensor(0.7750, device='cuda:0')
0.0611
0.0767
0.0841
0.1067
corrupt label
0.0968
acc  tensor(0.7659, device='cuda:0')
0.1193
0.0741
corrupt label
0.0537
0.0573
0.0954
acc  tensor(0.7672, device='cuda:0')
0.0509
0.0295
0.1422
0.1095
corrupt label
0.0568
acc  tensor(0.7649, device='cuda:0')
0.0509
corrupt label
0.0890
0.0769
0.0355
0.0514
acc  tensor(0.7788, device='cuda:0')
0.0365
0.0511
0.0927
0.1218
0.0756
corrupt label
acc  

In [27]:
correct, total = 0, 0
for idx, (data, labels, _) in enumerate(test_loader):
    data, labels = data.cuda(), labels.cuda()
    with torch.no_grad():
        x, _, _, _ = vae(data)
    preds = full_model(x)
    correct += (preds.argmax(dim=1) == labels).sum()
    total += data.shape[0]
print(correct / total)

tensor(0.6086, device='cuda:0')
